Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

<!--NAVIGATION-->
< [Side-Chain Packing](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.02-Side-chain-packing.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Docking](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/07.00-Docking.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# Design
Keywords: generate_resfile_from_pdb(), generate_resfile_from_pose(), create_packer_task(), mutate_residue()

In [ ]:
# Notebook setup
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.setup()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

**Make sure you are in the directory with the pdb files:**

`cd google_drive/My\ Drive/student-notebooks/`

In [ ]:
# From previous section:
from pyrosetta import *
from pyrosetta.teaching import *
pyrosetta.init()
pose = pose_from_pdb("inputs/1YY8.clean.pdb")
start_pose = Pose()
start_pose.assign(pose)
scorefxn = get_fa_scorefxn()

Design calculations can be accomplished simply by packing side chains with a rotamer set that includes all amino acid types. That is, when the Monte Carlo routine swaps rotamers, it could replace the existing side chain with another conformation of the same residue or some conformation of a different residue type. Trial mutations are accepted or rejected with the Metropolis criterion, and the standard full-atom energy function is supplemented by a reference energy term, `ref`, which represents the relative energies of each residue type in an unfolded peptide.

Design operations are easiest to specify through a data file called a “resfile.” You can create a resfile for a given pdb file or pose using the following toolbox methods:


```
from pyrosetta.toolbox import generate_resfile_from_pdb
generate_resfile_from_pdb("1YY8.clean.pdb", "1YY8.resfile")
from pyrosetta.toolbox import generate_resfile_from_pose
generate_resfile_from_pose(pose, "1YY8.resfile")
```

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Inside the resfile you will see:

```
NATAA
USE_INPUT_SC
start
```

This means that all of the native amino acids will remain the same, but we will allow repacking to other rotamers. You can change "NATAA" to any of the phrases in the table below.

Also, under "start", you can add exceptions for certain amino acids. Let's do an example.

| Name |  Definition  |
|------|------|
|  NATRO | use native amino acid and native rotamer (does not repack)|
|  NATAA | use native amino acid but allow repacking to other rotamers|
|  PIKAA ILV | use only the following amino acids (ILV) and allow repacking between them|
|  ALLAA | use all amino acids and all repacking|

## In your terminal:


Edit the resfile to force residue 49 to be glutamic acid (`49 A PIKAA E`) and ensure all other residues cannot be redesigned (change `NATAA` to `NATRO`). Save the file as `1YY8-K49E.resfile`.

Your resfile should look like this:
```
NATRO
USE_INPUT_SC
start
49 A PIKAA E
```

## Back here in the notebook:

Create a new task for design from the resfile:
```
from pyrosetta.rosetta.core.pack.task import TaskFactory, parse_resfile
task_design = TaskFactory.create_packer_task(pose)
parse_resfile(pose, task_design, "1YY8-K49E.resfile")
```

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Score the original `start_pose` conformation from the 1YY8 pdb for reference. Create a new `PackResiduesMover` with the `task_design` and use it to mutate residue 49 to glutamic acid. Confirm you mutated the residue by printing residue 49.

__Question:__ What is the predicted Δ*G* of the mutation? Is this a stabilizing mutation?

```
pose.assign(start_pose)
pack_mover = PackRotamersMover(scorefxn, task_design)
print(pose.residue(49))
pack_mover.apply(pose)
print(pose.residue(49))
print(scorefxn(pose) - scorefxn(start_pose))
```

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Note the residue reference energy term (`ref`) in the scoring function.

__Question:__ What is this value before and after you mutated the residue? What does this energy represent?

## In your terminal:

Create a new resfile that allows residue 49 to be designed freely (`49 A ALLAA`) and call it `1YY8-K49All.resfile`.

## Back here in the notebook:

Create a new `PackerTask` and `PackRotamersMover` and apply it.

__Question:__ What residue does Rosetta choose? Why?

```
pose.assign(start_pose)
task_design_all = TaskFactory.create_packer_task(pose)
parse_resfile(pose, task_design_all, "1YY8-K49All.resfile")
pack_mover_all = PackRotamersMover(scorefxn, task_design_all)
pack_mover_all.apply(pose)
print(pose.residue(49))
```

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Create your own resfile that will restrict residue 49 to only negatively charged residues using the resfile line `49 A PIKAA DE` and re-apply the design mover.

__Question:__ Now what residue is chosen? What is the new residue energy, and why (physically) is it less favorable than the last design?

Let’s try to make this design more favorable. Select several surrounding residues for design, and set them also to enable mutations to any residue. Call the design mover again.

__Question:__ Now what do you find?

It should be noted that PyRosetta includes a handy toolbox method mutate_residue() that will change a specified residue in a given pose into another. However, the rotamer of this new residue will not be optimized. For example:

```
from pyrosetta.toolbox import mutate_residue
pose.assign(start_pose)
print(pose.residue(49))
mutate_residue(pose, 49, 'E')
print(pose.residue(49))
```

## Programming Exercises


- *Refinement and discrimination*. Download the “single misfold” decoy set from the Decoys ’R Us repository at dd.compbio.washington.edu/ddownload.cgi?misfold. (Documentation for this project is at dd.compbio.washington.edu.) This repository has a single “correct” and “incorrect” predicted structure for several proteins. For this exercise, analyze pdbs 2CI2 and 2CRO; each has two “incorrect” structures offered. (Technical note: These decoys have an empty occupancy field in the PDB *ATOM* records; a value of 1 needs to be added before Rosetta will load these structures.)

    Write a program that will calculate and output the score for each decoy (i) as is from the PDB file, (ii) after packing only, (iii) after minimization only, and (iv) after packing and minimizing. For each of the four cases, compare the scores of the “correct” structure with those of the “incorrect” structure. Which schemes successfully discriminate the correct structures?


- Write a refinement protocol that will iterate between side-chain packing, small and shear moves, and minimization. Where is the best place to position the Monte Carlo acceptance test? Test your protocol by making 10 independently-refined structures for the correct and incorrect decoys of 2CRO from the Decoys ’R Us single misfold set. Is this protocol able to discriminate the correct decoy? Submit your code.


- HIV-1 protease is a major drug target for antiretroviral therapies. Protease inhibitors are designed from substrate peptide mimics. We will attempt to take a natural substrate peptide of HIV-1 protease and design it for improved binding — potentially to serve as a good template for drug design. Use PDB file 1KJG for the following analysis.
    
    
    - Turn on side-chain packing for the protease active site (residues 8, 23, 25, 29, 30, 32, 45, 47, 50, 53, 82, and 84 of both chains A and B) and for the peptide (residues 2–9 on chain P; all of these numbers follow the PDB numbering).


    - Repack the above side chains and then energy minimize those same side chains with the backbone fixed. Generate 10 decoys and record the energies for each decoy. This will represent the reference state: the wild-type peptide bound to the protease.


    - For residue 2 of the peptide (chain P), allow repacking to any of the 20 amino acid residues, while leaving the packing and side-chain minimization the same as in step b. Generate 10 decoys and record the energies. These will represent single mutants at that residue position.


    - Repeat step c for each of the other 8 residues in the substrate peptide.
    
    
    - Take the lowest energy for each mutation position and compare it to the lowest energy for the wild type. Do single mutants at any of these positions improve the energy over the wild type? Which ones? By how much? Which energy components are mostly responsible?
    

    - Which peptide residue positions are easiest to improve? Which positions are the hardest?


    - Are there any other trends? Hydrophobic vs. polar, bulky residues vs. small residues, etc.?


    - Altman et al. (Proteins 2008) found, using their own computational design algorithm, that the most favorable sequences were a triple mutant E3D/T4I/V6L, a single mutant T4V, and a single mutant E3Q. How do their results compare with yours?


    - Natural substrates are often sub-optimal binders. Why would this be advantageous?


- Effect of backbone conformation on design. HIV-1 protease is promiscuous, meaning it can cleave a wide range of peptides beyond the ten natural substrates of the virus. Let’s examine the preferences of the enzyme through Rosetta design calculations.

    - Download HIV-1 protease in complex with CA-P2 peptide (1F7A). Select the eight peptide residues for unrestricted design and let Rosetta redesign the substrate sequence. What is the new sequence and how does it compare to the original? What percent of the original sequence was optimal for its structure?


    - Download HIV-1 protease in complex with RT-RH peptide (1KJG). (Note that the enzyme is the same here, but it is crystallized with a different substrate.) Again, design the eight substrate residues with Rosetta. What percent of this substrate sequence is optimal for this crystal structure? ____%


    - How do the designed sequences of (a) and (b) compare? Why should they be the same? Why would they not be the same? What are the implications for the field of computational protein design?


- Write a program which iterates between design of all residues of a protein and refinement via small, shear, and minimization moves.


## Thought Question

What is the thermodynamic meaning of the ref energy term, and what does it correspond to physically?
During evolution, the genome sequence may mutate to cause protein sequence changes. Alternately, one could consider the difference in evolutionary propensities for each residue type. How could you derive reference energies from sequence data, and what would that mean? 


How do Kuhlman & Baker fit the reference energies in their 2000 PNAS paper?


## References


- S. C. Lovell et al., “The penultimate rotamer library,” Proteins 40, 389-408 (2000).


- R. L. Dunbrack & F. E. Cohen, “Bayesian statistical analysis of protein side-chain rotamer preferences,” Protein Sci. 6, 1661-1681 (1997)

<!--NAVIGATION-->
< [Side-Chain Packing](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.02-Side-chain-packing.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Docking](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/07.00-Docking.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>